In [2]:
import tkinter as tk
from tkinter import Canvas, Button, Label, filedialog, messagebox, PhotoImage, ROUND
import PIL
from PIL import Image, ImageDraw
import cv2
import joblib as jb
import pandas as pd


class RandomForestPage(tk.Toplevel):
    def __init__(self, master=None, home_page=None, model_type="RF"):
        super().__init__(master)
        self.home_page = home_page  # Store the home page reference
        self.title(f"{model_type} Page")
        self.model_type = model_type  # Store the model type

        # Add widgets and customize the appearance of the Random Forest page as needed

        self.char_map = pd.read_csv('Dataset/emnist-letters-mapping.txt', delimiter=' ', header=None, index_col=0, squeeze=True)
        self.mapping = {index: chr(label) for index, label in enumerate(self.char_map)}

        if model_type == "RF":
            self.model = jb.load("RF.sav")
        elif model_type == "DT":
            self.model = jb.load("DT.sav")

        self.lastx, self.lasty = None, None
        self.image1 = PIL.Image.new('RGB', (168, 168), 'black')
        self.draw = ImageDraw.Draw(self.image1)

        self.cv = Canvas(self, width=168, height=168, bg='black')
        self.cv.bind('<1>', self.activate_paint)
        self.cv.pack(expand=tk.YES, fill=tk.BOTH)

        self.btn_save = Button(self, text=f"Predict ({model_type})", command=self.recognize, relief=tk.FLAT,
                               bg="#4CAF50", fg="white", activebackground="#45a049")
        self.btn_save.pack(pady=5, padx=10)

        self.btn_clr = Button(self, text="Clear", command=self.clear, relief=tk.FLAT, bg="#f44336", fg="white",
                              activebackground="#d32f2f")
        self.btn_clr.pack(pady=5, padx=10)

        self.lb = Label(self, text="Letter: ")
        self.lb.pack()

        # Prediction area
        self.prediction_area = tk.Frame(self, width=200, height=50, bg='black')
        self.prediction_area.pack(pady=10, padx=10)
        self.prediction_label = Label(self.prediction_area, text="", font=("Helvetica", 24), fg="white", bg="black")
        self.prediction_label.pack(expand=tk.YES)

        # Accuracy label
        self.accuracy_label = Label(self, text="", font=("Helvetica", 16), fg="black", bg="lightgray")
        self.accuracy_label.pack(pady=5)

        # Upload Image Button
        self.upload_image_button = Button(self, text="Upload Image", command=self.upload_image_action,
                                          relief=tk.FLAT, bg="#2196F3", fg="white", activebackground="#1976D2")
        self.upload_image_button.pack(pady=5, padx=10)

        # Back to Home Page Button
        self.back_button = Button(self, text="Back to Home Page", command=self.back_to_home_page,
                                  relief=tk.FLAT, bg="#607D8B", fg="white", activebackground="#455A64")
        self.back_button.pack(pady=5, padx=10)

    def resize_img(self, cimage):
        BW = cv2.cvtColor(cimage, cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(BW, (28, 28), interpolation=cv2.INTER_AREA)
        return resized_img

    def activate_paint(self, e):
        self.cv.bind('<B1-Motion>', self.paint)
        self.lastx, self.lasty = e.x, e.y

    def paint(self, e):
        x, y = e.x, e.y
        self.cv.create_line((self.lastx, self.lasty, x, y), width=20, fill='white', capstyle=ROUND, smooth=False,
                            splinesteps=24)
        self.draw.line((self.lastx, self.lasty, x, y), fill='white', width=20)
        self.lastx, self.lasty = x, y

    def clear(self):
        self.image1 = PIL.Image.new('RGB', (168, 168), 'black')
        self.draw = ImageDraw.Draw(self.image1)
        self.cv.delete("all")

    def recognize(self):
        self.image1.save('image.png')
        im = cv2.imread('image.png')
        final = self.resize_img(im)
        final = final.reshape(-1)

        # Use predict_proba instead of predict
        probabilities = self.model.predict_proba([final])[0]
        y_pred = self.model.predict([final])
        predicted_letter = self.mapping[(y_pred[0]) - 1]

        # Find the class index with the highest probability
        predicted_class = probabilities.argmax()

        # Display the predicted letter and confidence score
        self.lb.config(text=f"Letter: {predicted_letter}")
        confidence_score = probabilities[predicted_class]
        self.accuracy_label.config(text=f"Confidence: {confidence_score:.2%}")

        # Update the prediction area background color based on the predicted letter
        bg_color = 'green' if predicted_letter.isupper() else 'blue'
        self.prediction_area.configure(bg=bg_color)
        self.prediction_label.config(text=predicted_letter)

    def upload_image_action(self):
        file_path = filedialog.askopenfilename(title=f"Select an Image File for {self.model_type}")
        if file_path:
            self.show_action_message(f"Selected Image ({self.model_type}): {file_path}")
        else:
            self.show_action_message(f"No image selected for {self.model_type}.")

    def show_action_message(self, message):
        messagebox.showinfo("Action", message)

    def back_to_home_page(self):
        # Destroy the current window and show the home page window
        self.destroy()
        self.home_page.deiconify()


class DecisionTreePage(tk.Toplevel):
    def __init__(self, master=None, home_page=None, model_type="DT"):
        super().__init__(master)
        self.home_page = home_page  # Store the home page reference
        self.title(f"{model_type} Page")
        self.model_type = model_type  # Store the model type

        # Add widgets and customize the appearance of the Decision Tree page as needed

        self.char_map = pd.read_csv('Dataset/emnist-letters-mapping.txt', delimiter=' ', header=None, index_col=0,
                                    squeeze=True)
        self.mapping = {index: chr(label) for index, label in enumerate(self.char_map)}

        self.model = jb.load("DT.sav")

        self.lastx, self.lasty = None, None
        self.image1 = PIL.Image.new('RGB', (168, 168), 'black')
        self.draw = ImageDraw.Draw(self.image1)

        self.cv = Canvas(self, width=168, height=168, bg='black')
        self.cv.bind('<1>', self.activate_paint)
        self.cv.pack(expand=tk.YES, fill=tk.BOTH)

        self.btn_save = Button(self, text=f"Predict ({model_type})", command=self.recognize, relief=tk.FLAT,
                               bg="#4CAF50", fg="white", activebackground="#45a049")
        self.btn_save.pack(pady=5, padx=10)

        self.btn_clr = Button(self, text="Clear", command=self.clear, relief=tk.FLAT, bg="#f44336", fg="white",
                              activebackground="#d32f2f")
        self.btn_clr.pack(pady=5, padx=10)

        self.lb = Label(self, text="Letter: ")
        self.lb.pack()

        # Prediction area
        self.prediction_area = tk.Frame(self, width=200, height=50, bg='black')
        self.prediction_area.pack(pady=10, padx=10)
        self.prediction_label = Label(self.prediction_area, text="", font=("Helvetica", 24), fg="white", bg="black")
        self.prediction_label.pack(expand=tk.YES)

        # Accuracy label
        self.accuracy_label = Label(self, text="", font=("Helvetica", 16), fg="black", bg="lightgray")
        self.accuracy_label.pack(pady=5)

        # Upload Image Button
        self.upload_image_button = Button(self, text="Upload Image", command=self.upload_image_action,
                                          relief=tk.FLAT, bg="#2196F3", fg="white", activebackground="#1976D2")
        self.upload_image_button.pack(pady=5, padx=10)

        # Back to Home Page Button
        self.back_button = Button(self, text="Back to Home Page", command=self.back_to_home_page,
                                  relief=tk.FLAT, bg="#607D8B", fg="white", activebackground="#455A64")
        self.back_button.pack(pady=5, padx=10)

    def resize_img(self, cimage):
        BW = cv2.cvtColor(cimage, cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(BW, (28, 28), interpolation=cv2.INTER_AREA)
        return resized_img

    def activate_paint(self, e):
        self.cv.bind('<B1-Motion>', self.paint)
        self.lastx, self.lasty = e.x, e.y

    def paint(self, e):
        x, y = e.x, e.y
        self.cv.create_line((self.lastx, self.lasty, x, y), width=20, fill='white', capstyle=ROUND, smooth=False,
                            splinesteps=24)
        self.draw.line((self.lastx, self.lasty, x, y), fill='white', width=20)
        self.lastx, self.lasty = x, y

    def clear(self):
        self.image1 = PIL.Image.new('RGB', (168, 168), 'black')
        self.draw = ImageDraw.Draw(self.image1)
        self.cv.delete("all")

    def recognize(self):
        self.image1.save('image.png')
        im = cv2.imread('image.png')
        final = self.resize_img(im)
        final = final.reshape(-1)

        # Use predict_proba instead of predict
        probabilities = self.model.predict_proba([final])[0]
        y_pred = self.model.predict([final])
        predicted_letter = self.mapping[(y_pred[0]) - 1]

        # Find the class index with the highest probability
        predicted_class = probabilities.argmax()

        # Display the predicted letter and confidence score
        self.lb.config(text=f"Letter: {predicted_letter}")
        confidence_score = probabilities[predicted_class]
        self.accuracy_label.config(text=f"Confidence: {confidence_score:.2%}")

        # Update the prediction area background color based on the predicted letter
        bg_color = 'green' if predicted_letter.isupper() else 'blue'
        self.prediction_area.configure(bg=bg_color)
        self.prediction_label.config(text=predicted_letter)

    def upload_image_action(self):
        file_path = filedialog.askopenfilename(title=f"Select an Image File for {self.model_type}")
        if file_path:
            self.show_action_message(f"Selected Image ({self.model_type}): {file_path}")
        else:
            self.show_action_message(f"No image selected for {self.model_type}.")

    def show_action_message(self, message):
        messagebox.showinfo("Action", message)

    def back_to_home_page(self):
        # Destroy the current window and show the home page window
        self.destroy()
        self.home_page.deiconify()


class HomePageGUI:
    def __init__(self, master):
        self.master = master
        master.title("Home Page")
        master.geometry("300x300")
        master.configure(bg="lightgray")  # Set background color for the main window

        # Welcome Statement
        self.label = tk.Label(master, text="Recognize your Letter!", font=("Helvetica", 16), bg="lightgray")
        self.label.pack(pady=20)

        # Choose Action Statement
        self.choose_label = tk.Label(master, text="Please choose a model:", font=("Helvetica", 12), bg="lightgray")
        self.choose_label.pack()

        # Decision Tree Button
        self.decision_tree_button = tk.Button(master, text="Decision Tree", command=self.decision_tree_action,
                                              relief=tk.FLAT, bg="#FFC107", fg="black", activebackground="#FFA000")
        self.decision_tree_button.pack(pady=10, padx=10)

        # Random Forest Button
        self.random_forest_button = tk.Button(master, text="Random Forest", command=self.random_forest_action,
                                              relief=tk.FLAT, bg="#FF5722", fg="white", activebackground="#E64A19")
        self.random_forest_button.pack(pady=10, padx=10)

    def decision_tree_action(self):
        # Hide the home page window
        self.master.withdraw()
        decision_tree_page = DecisionTreePage(self.master, home_page=self.master, model_type="DT")

    def random_forest_action(self):
        # Hide the home page window
        self.master.withdraw()
        random_forest_page = RandomForestPage(self.master, home_page=self.master, model_type="RF")


if __name__ == "__main__":
    root = tk.Tk()
    app = HomePageGUI(root)
    root.mainloop()


C:\Users\mohamed hanafy\AppData\Local\Temp\ipykernel_17920\3311611060.py:19: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  self.char_map = pd.read_csv('Dataset/emnist-letters-mapping.txt', delimiter=' ', header=None, index_col=0, squeeze=True)
C:\Users\mohamed hanafy\AppData\Local\Temp\ipykernel_17920\3311611060.py:19: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  self.char_map = pd.read_csv('Dataset/emnist-letters-mapping.txt', delimiter=' ', header=None, index_col=0, squeeze=True)
C:\Users\mohamed hanafy\AppData\Local\Temp\ipykernel_17920\3311611060.py:19: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  self.char_map = pd.read_csv('Dataset/emnist-letters-mapping.txt', delim

KeyboardInterrupt: 

KeyboardInterrupt: 